# Customer Churn Prediction :

Churn rate is a marketing metric that describes the number of customers who leave a business over a specific time period. Every user is assigned a prediction value that estimates their state of churn at any given time.

In this notebook I have performed exploratory data ananlysis on the dataset. This dataset mainly consists numerical and categorical feaatures.

# Libraries

In [44]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from feature_engine.encoding import MeanEncoder


In [45]:
# importing dataset
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [46]:
print("The shape of the Training dataset is", train.shape)
print("The shape of the Testing dataset is", test.shape)

The shape of the Training dataset is (36992, 25)
The shape of the Testing dataset is (19919, 24)


In [47]:
train.sample(5)


,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
3621,fffe43004900440036003200320037003900,Alyson Ericksen,43,F,2HGN2O3,City,Premium Membership,2015-10-17,Yes,CID57870,...,177.630000,12713.56,13.0,760.620000,Yes,No,No,Not Applicable,Poor Product Quality,3
25815,fffe43004900440035003400330033003800,Candy Mast,19,F,KDU0Q48,Town,Basic Membership,2016-10-29,?,CID22363,...,481.410000,43802.98,21.0,1141.957842,No,Yes,Yes,No Information Available,Poor Product Quality,5
36089,fffe43004900440035003900300035003000,Antonio Bodine,40,M,MWKSLB6,Town,No Membership,2017-10-07,?,CID42765,...,641.690000,33666.00,18.0,557.476478,Yes,No,No,Not Applicable,Poor Website,5
30114,fffe43004900440034003300390033003100,Luetta Hullinger,12,F,NYP25LP,City,Gold Membership,2017-03-31,No,xxxxxxxx,...,363.420000,29077.92,27.0,NaN,Yes,Yes,Yes,Unsolved,Poor Product Quality,3
15505,fffe43004900440031003500390032003900,Samantha Ellenwood,58,M,3IO2WV9,Village,Basic Membership,2016-12-22,Yes,CID26206,...,1614.849289,40418.49,21.0,139.185375,Yes,No,Yes,Solved in Follow-up,Too many ads,5


In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   36992 non-null  object 
 1   Name                          36992 non-null  object 
 2   age                           36992 non-null  int64  
 3   gender                        36992 non-null  object 
 4   security_no                   36992 non-null  object 
 5   region_category               31564 non-null  object 
 6   membership_category           36992 non-null  object 
 7   joining_date                  36992 non-null  object 
 8   joined_through_referral       36992 non-null  object 
 9   referral_id                   36992 non-null  object 
 10  preferred_offer_types         36704 non-null  object 
 11  medium_of_operation           36992 non-null  object 
 12  internet_option               36992 non-null  object 
 13  l

**About Features**
- *customer_id* - Represents the unique identification number of a customer

- *Name* - Represents the name of a customer

- *age - Represents the age of a customer

- *security_no* - Represents a unique security number that is used to identify a person

- *region_category* - Represents the region that a customer belongs to

- *membership_category* - Represents the category of the membership that a customer is using

- *joining_date* - Represents the date when a customer became a member

- *joined_through_referral* - Represents whether a customer joined using any referral code or ID

- *referral_id* - Represents a referral ID

- *preferred_offer_types* - Represents the type of offer that a customer prefers

- *medium_of_operation* - Represents the medium of operation that a customer uses for transactions

- *internet_option* - Represents the type of internet service a customer uses

- *last_visit_time* - Represents the last time a customer visited the website

- *days_since_last_login* - Represents the no. of days since a customer last logged into the website

- *avg_time_spent* - Represents the average time spent by a customer on the website

- *avg_transaction_value* - Represents the average transaction value of a customer

- *avg_frequency_login_days* - Represents the no. of times a customer has logged in to the website

- *points_in_wallet* - Represents the points awarded to a customer on each transaction

- *used_special_discount* - Represents whether a customer uses special discounts offered

- *offer_application_preference* - Represents whether a customer prefers offers

- *past_complaint* - Represents whether a customer has raised any complaints

- *complaint_status* - Represents whether the complaints raised by a customer was resolved

- *feedback* - Represents the feedback provided by a customer

- *churn_risk_score* - Represents the churn risk score that ranges from 1 to 5

## Basic Statistics and Data wrangling

In [49]:
train.describe()


,age,days_since_last_login,avg_time_spent,avg_transaction_value,points_in_wallet,churn_risk_score
count,36992.000000,36992.000000,36992.000000,36992.000000,33549.000000,36992.000000
mean,37.118161,-41.915576,243.472334,29271.194003,686.882199,3.463397
std,15.867412,228.819900,398.289149,19444.806226,194.063624,1.409661
min,10.000000,-999.000000,-2814.109110,800.460000,-760.661236,-1.000000
25%,23.000000,8.000000,60.102500,14177.540000,616.150000,3.000000
50%,37.000000,12.000000,161.765000,27554.485000,697.620000,4.000000
75%,51.000000,16.000000,356.515000,40855.110000,763.950000,5.000000
max,64.000000,26.000000,3235.578521,99914.050000,2069.069761,5.000000


### About Data
- Most of the people have age around 37.
- As time can not be negative so negative values in column "days_since_last_login","avg_time_spent" are outliers.
- Similarly acc. balance can not be negative but in "points_in_wallet" column have and can be considers as outlier.
- churn_risk_score is out target column and it contain negative value which may be missrecorded observation. 

## Finding null values if any

In [50]:
train.isnull().sum()

customer_id                        0
Name                               0
age                                0
gender                             0
security_no                        0
region_category                 5428
membership_category                0
joining_date                       0
joined_through_referral            0
referral_id                        0
preferred_offer_types            288
medium_of_operation                0
internet_option                    0
last_visit_time                    0
days_since_last_login              0
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days           0
points_in_wallet                3443
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
churn_risk_score                   0
dtype: int64

In [51]:
# Percentage of missing data
def percent_missing(df):
    missing_values_count = df.isnull().sum()
    total_cells=np.product(df.shape)
    total_missing=missing_values_count.sum()
    return (total_missing/total_cells)*100

print('Percent of data that is missing from train data:-',percent_missing(train))
print('Percent of data that is missing from test data:-',percent_missing(test))



Percent of data that is missing from train data:- 0.9903762975778546
Percent of data that is missing from test data:- 1.0605452080927757


In [52]:
# Seprating features
def features(df):
    return df.select_dtypes(include=np.number).columns.tolist(),df.select_dtypes(exclude=np.number).columns.tolist()



In [53]:
def preprocessing(df):
    df.gender = df.gender.replace('Unknown', np.nan)
    df.joined_through_referral = df.joined_through_referral.replace('?', np.nan)
    df.medium_of_operation = df.medium_of_operation.replace('?', np.nan)

    df.region_category.fillna('Village', axis=0, inplace=True)
    df.preferred_offer_types.fillna(df.preferred_offer_types.mode()[0], axis=0, inplace=True)
    df.points_in_wallet.fillna(value=df.points_in_wallet.median(), axis=0, inplace=True)
    df.gender.fillna(df.gender.mode()[0], axis=0, inplace=True)
    df.joined_through_referral.fillna(df.joined_through_referral.mode()[0], axis=0, inplace=True)
    df.medium_of_operation.fillna('Both', axis=0, inplace=True)

    df['avg_frequency_login_days'] = df['avg_frequency_login_days'].apply(lambda x: 0 if x == 'Error' else x)
    df['avg_frequency_login_days'] = pd.to_numeric(df['avg_frequency_login_days'])
    df['avg_frequency_login_days'] = df['avg_frequency_login_days'].apply(lambda x: 0 if x < 0 else x)
    df['avg_transaction_value'] = df['avg_transaction_value'].apply(lambda x: 0 if x < 0 else x)
    df['days_since_last_login'] = df['days_since_last_login'].apply(lambda x: 0 if x < 0 else x)
    df['avg_frequency_login_days'] = df['avg_frequency_login_days'].apply(lambda x: 0 if x < 0 else x)
    df['membership_by_refer_id_min'] = df.groupby('referral_id')['membership_category'].transform('min')
    df['membership_by_refer_id_max'] = df.groupby('referral_id')['membership_category'].transform('max')
    return df

In [54]:
train=preprocessing(train)
test=preprocessing(test)

In [55]:
train

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,membership_by_refer_id_min,membership_by_refer_id_max
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,...,17.0,781.750000,Yes,Yes,No,Not Applicable,Products always in Stock,2,Basic Membership,Silver Membership
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,2017-08-28,No,CID21329,...,10.0,697.620000,Yes,No,Yes,Solved,Quality Customer Care,1,Basic Membership,Premium Membership
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,...,22.0,500.690000,No,Yes,Yes,Solved in Follow-up,Poor Website,5,No Membership,Platinum Membership
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,...,6.0,567.660000,No,Yes,Yes,Unsolved,Poor Website,5,No Membership,No Membership
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,...,16.0,663.060000,No,Yes,Yes,Solved,Poor Website,5,Basic Membership,Silver Membership
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,fffe43004900440035003500390036003100,Cuc Tarr,46,F,6F51HFO,Village,Basic Membership,2017-09-21,No,xxxxxxxx,...,6.0,639.510000,No,Yes,Yes,No Information Available,No reason specified,4,Basic Membership,Silver Membership
36988,fffe43004900440033003500380036003600,Jenni Stronach,29,F,21KSM8Y,Town,Basic Membership,2016-06-27,No,xxxxxxxx,...,28.0,527.990000,Yes,No,No,Not Applicable,Poor Customer Service,5,Basic Membership,Silver Membership
36989,fffe4300490044003500330034003100,Luciana Kinch,23,F,XK1IM9H,Village,Basic Membership,2016-09-11,Yes,CID3838,...,0.0,680.470000,No,Yes,Yes,Unsolved,Poor Website,4,Basic Membership,Premium Membership
36990,fffe43004900440031003200390039003000,Tawana Ardoin,53,M,K6VTP1Z,Village,Platinum Membership,2017-06-15,No,xxxxxxxx,...,20.0,197.264414,Yes,Yes,No,Not Applicable,No reason specified,3,Basic Membership,Silver Membership


In [56]:
numerics_features,categorical_features=features(train)


['age',
 'days_since_last_login',
 'avg_time_spent',
 'avg_transaction_value',
 'avg_frequency_login_days',
 'points_in_wallet',
 'churn_risk_score',
 'day',
 'year',
 'month',
 'visiting_hour',
 'visiting_min',
 'visiting_sec',
 'first_name',
 'hours_since_last_login',
 'actual_time_spent',
 'customer_by_membership',
 'points_by_Name',
 'avg_trans_by_security_no',
 'trans_points_add',
 'trans_points_sub']

In [57]:
# Finding column tha have negative values
def egn(df):
    # reemoving negative values
    return df[(df[numerics_features] > 0).all(1)]
egn(train[numerics_features])

,age,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,churn_risk_score
0,18,17,300.63,53005.25,17.0,781.750000,2
1,32,16,306.34,12838.38,10.0,697.620000,1
2,44,14,516.16,21027.00,22.0,500.690000,5
3,37,11,53.27,25239.56,6.0,567.660000,5
4,31,20,113.13,24483.66,16.0,663.060000,5
...,...,...,...,...,...,...,...
36983,45,9,49.33,45358.49,11.0,242.979625,5
36984,51,24,312.33,63446.71,2.0,778.700000,1
36985,12,13,418.38,56397.21,7.0,725.890000,2
36986,27,13,135.83,8225.68,16.0,748.570000,3


In [58]:
# all unique values
def finding_uniques(df,feature):
    """
    finding all the unique elements and ploting them
    """
    return df[feature].value_counts(),sns.countplot(df[feature], hue = df['churn_risk_score'])

In [59]:
train.isna().sum()

customer_id                     0
Name                            0
age                             0
gender                          0
security_no                     0
region_category                 0
membership_category             0
joining_date                    0
joined_through_referral         0
referral_id                     0
preferred_offer_types           0
medium_of_operation             0
internet_option                 0
last_visit_time                 0
days_since_last_login           0
avg_time_spent                  0
avg_transaction_value           0
avg_frequency_login_days        0
points_in_wallet                0
used_special_discount           0
offer_application_preference    0
past_complaint                  0
complaint_status                0
feedback                        0
churn_risk_score                0
membership_by_refer_id_min      0
membership_by_refer_id_max      0
dtype: int64

# Creating features


In [60]:
def feature_engg(df):
    df['joining_date'] = pd.to_datetime(df['joining_date'], format='%Y-%m-%d')
    df['day'] = df['joining_date'].dt.day
    df['year'] = df['joining_date'].dt.year
    df['month'] = df['joining_date'].dt.month
    df = df.drop('joining_date', axis=1)

    df['last_visit_time'] = pd.to_datetime(df['last_visit_time'], format='%H:%M:%S')
    df['visiting_hour'] = df['last_visit_time'].dt.hour
    df['visiting_min'] = df['last_visit_time'].dt.minute
    df['visiting_sec'] = df['last_visit_time'].dt.second
    df = df.drop('last_visit_time', axis=1)

    df['first_name'] = df.Name.str.split(' ', expand=True)[0]
    naming = df.first_name.value_counts(normalize=True).to_dict()
    df.first_name = df.first_name.map(naming)
    df['hours_since_last_login'] = df.days_since_last_login * 24
    df['actual_time_spent'] = df.avg_time_spent - df.visiting_sec

    df['customer_by_membership'] = df.groupby('membership_category')['customer_id'].transform('count')
    df['points_by_Name'] = df.groupby('points_in_wallet')['Name'].transform('count')
    df['avg_trans_by_security_no'] = df.groupby('avg_frequency_login_days')['security_no'].transform('count')

    df['trans_points_add'] = df['avg_transaction_value'] + df['points_in_wallet']
    df['trans_points_sub'] = df['avg_transaction_value'] - df['points_in_wallet']
    return df



In [61]:
train=feature_engg(train)
test=feature_engg(test)

In [62]:
train

,customer_id,Name,age,gender,security_no,region_category,membership_category,joined_through_referral,referral_id,preferred_offer_types,...,visiting_min,visiting_sec,first_name,hours_since_last_login,actual_time_spent,customer_by_membership,points_by_Name,avg_trans_by_security_no,trans_points_add,trans_points_sub
0,fffe4300490044003600300030003800,Pattie Morrisey,18,F,XW0DQ7H,Village,Platinum Membership,No,xxxxxxxx,Gift Vouchers/Coupons,...,8,2,0.002379,408,298.630000,4338,1,1349,53787.000000,52223.500000
1,fffe43004900440032003100300035003700,Traci Peery,32,F,5K0N3X1,City,Premium Membership,No,CID21329,Gift Vouchers/Coupons,...,38,13,0.001919,384,293.340000,4455,3446,1334,13536.000000,12140.760000
2,fffe4300490044003100390032003600,Merideth Mcmeen,44,F,1F2TCL3,Town,No Membership,Yes,CID12313,Gift Vouchers/Coupons,...,53,21,0.002730,336,495.160000,7692,1,1054,21527.690000,20526.310000
3,fffe43004900440036003000330031003600,Eufemia Cardwell,37,M,VJGJ33N,City,No Membership,Yes,CID3793,Gift Vouchers/Coupons,...,57,50,0.000622,264,3.270000,7692,1,1336,25807.220000,24671.900000
4,fffe43004900440031003900350030003600,Meghan Kosak,31,F,SVZXCWB,City,No Membership,No,xxxxxxxx,Credit/Debit Card Offers,...,46,44,0.002217,480,69.130000,7692,3,1319,25146.720000,23820.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,fffe43004900440035003500390036003100,Cuc Tarr,46,F,6F51HFO,Village,Basic Membership,No,xxxxxxxx,Credit/Debit Card Offers,...,14,5,0.002622,48,-655.682759,7724,1,1336,27917.190000,26638.170000
36988,fffe43004900440033003500380036003600,Jenni Stronach,29,F,21KSM8Y,Town,Basic Membership,No,xxxxxxxx,Without Offers,...,18,31,0.002974,312,-669.123421,7724,2,1107,11597.700000,10541.720000
36989,fffe4300490044003500330034003100,Luciana Kinch,23,F,XK1IM9H,Village,Basic Membership,Yes,CID3838,Gift Vouchers/Coupons,...,50,25,0.002082,288,129.940000,7724,3,4205,38808.030000,37447.090000
36990,fffe43004900440031003200390039003000,Tawana Ardoin,53,M,K6VTP1Z,Village,Platinum Membership,No,xxxxxxxx,Gift Vouchers/Coupons,...,50,3,0.001162,360,479.610000,4338,1,1069,2576.124414,2181.595586


In [63]:
# droping customer id
train.drop(['security_no'], axis = 1,inplace=True)
test.drop(['security_no'], axis = 1,inplace=True)
train.drop(['Name'], axis = 1,inplace=True)
test.drop(['Name'], axis = 1,inplace=True)



In [64]:
# seprating categorical and numerical 
numerical,categorical=features(train)


In [65]:
categorical.remove('customer_id')

In [66]:
train['avg_frequency_login_days'].value_counts()

0.000000     4205
13.000000    1394
19.000000    1365
8.000000     1361
14.000000    1355
             ... 
44.675779       1
42.302201       1
51.835579       1
31.241581       1
46.066445       1
Name: avg_frequency_login_days, Length: 971, dtype: int64

In [67]:
train['used_special_discount'].value_counts()

Yes    20342
No     16650
Name: used_special_discount, dtype: int64

In [68]:
train['offer_application_preference'].value_counts()

Yes    20440
No     16552
Name: offer_application_preference, dtype: int64

In [69]:
train['past_complaint'].value_counts()

No     18602
Yes    18390
Name: past_complaint, dtype: int64

In [70]:
train['feedback'].value_counts()

Poor Product Quality        6350
No reason specified         6290
Too many ads                6279
Poor Website                6271
Poor Customer Service       6252
Reasonable Price            1417
User Friendly Website       1391
Products always in Stock    1382
Quality Customer Care       1360
Name: feedback, dtype: int64

In [71]:
train['complaint_status'].value_counts()

Not Applicable              18602
Unsolved                     4644
Solved                       4619
Solved in Follow-up          4577
No Information Available     4550
Name: complaint_status, dtype: int64

In [72]:
#Applying mean encoder on categorical values
target = train.churn_risk_score
mean_enc = MeanEncoder(variables= categorical)
mean_enc.fit(train[categorical], target)
train[categorical] = mean_enc.transform(train[categorical])
test[categorical] = mean_enc.transform(test[categorical])

/home/amninder/.local/lib/python3.8/site-packages/feature_engine/encoding/base_encoder.py:141: UserWarning: NaN values were introduced in the returned dataframe by the encoder.This means that some of the categories in the input dataframe were not present in the training set used when the fit method was called. Thus, mappings for those categories do not exist. Try using the RareLabelCategoricalEncoder to remove infrequent categories before calling this encoder.
  warnings.warn(


In [73]:
train

,customer_id,age,gender,region_category,membership_category,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,...,visiting_min,visiting_sec,first_name,hours_since_last_login,actual_time_spent,customer_by_membership,points_by_Name,avg_trans_by_security_no,trans_points_add,trans_points_sub
0,fffe4300490044003600300030003800,18,3.463152,3.422731,2.304518,3.42459,3.407710,3.378492,3.437031,3.457343,...,8,2,0.002379,408,298.630000,4338,1,1349,53787.000000,52223.500000
1,fffe43004900440032003100300035003700,32,3.463152,3.476015,2.296521,3.42459,2.500000,3.378492,3.430101,3.464717,...,38,13,0.001919,384,293.340000,4455,3446,1334,13536.000000,12140.760000
2,fffe4300490044003100390032003600,44,3.463152,3.481172,4.494540,3.51594,3.666667,3.378492,3.430101,3.457343,...,53,21,0.002730,336,495.160000,7692,1,1054,21527.690000,20526.310000
3,fffe43004900440036003000330031003600,37,3.463645,3.476015,4.494540,3.51594,5.000000,3.378492,3.430101,3.464717,...,57,50,0.000622,264,3.270000,7692,1,1336,25807.220000,24671.900000
4,fffe43004900440031003900350030003600,31,3.463152,3.476015,4.494540,3.42459,3.407710,3.447531,3.514269,3.464717,...,46,44,0.002217,480,69.130000,7692,3,1319,25146.720000,23820.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,fffe43004900440035003500390036003100,46,3.463152,3.422731,4.470999,3.42459,3.407710,3.447531,3.430101,3.457343,...,14,5,0.002622,48,-655.682759,7724,1,1336,27917.190000,26638.170000
36988,fffe43004900440033003500380036003600,29,3.463152,3.481172,4.470999,3.42459,3.407710,3.568330,3.514269,3.457343,...,18,31,0.002974,312,-669.123421,7724,2,1107,11597.700000,10541.720000
36989,fffe4300490044003500330034003100,23,3.463152,3.422731,4.470999,3.51594,3.666667,3.378492,3.430101,3.457343,...,50,25,0.002082,288,129.940000,7724,3,4205,38808.030000,37447.090000
36990,fffe43004900440031003200390039003000,53,3.463645,3.422731,2.304518,3.42459,3.407710,3.378492,3.514269,3.464717,...,50,3,0.001162,360,479.610000,4338,1,1069,2576.124414,2181.595586


In [80]:
print(categorical)

['gender', 'region_category', 'membership_category', 'joined_through_referral', 'referral_id', 'preferred_offer_types', 'medium_of_operation', 'internet_option', 'used_special_discount', 'offer_application_preference', 'past_complaint', 'complaint_status', 'feedback', 'membership_by_refer_id_min', 'membership_by_refer_id_max']


In [74]:
train.to_csv('../input/proc_test.csv', index= False)
test.to_csv('../input/proc_test.csv', index= False)

In [75]:
test.head()

,customer_id,age,gender,region_category,membership_category,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,...,visiting_min,visiting_sec,first_name,hours_since_last_login,actual_time_spent,customer_by_membership,points_by_Name,avg_trans_by_security_no,trans_points_add,trans_points_sub
0,fffe43004900440031003700300030003400,50,3.463152,3.422731,2.296521,3.42459,3.40771,3.568330,3.514269,3.457343,...,19,30,0.002811,288,356.26,2365,1,743,41455.270000,39987.610000
1,fffe43004900440031003900370037003300,41,3.463645,3.422731,2.900074,3.42459,3.40771,3.568330,3.430101,3.468209,...,21,16,0.002811,264,21.80,3750,1,712,10370.400000,8918.400000
2,fffe43004900440034003800360037003000,31,3.463152,3.481172,3.186039,3.42459,3.40771,3.378492,3.437031,3.464717,...,40,39,0.000803,432,176.36,3199,1,581,4407.030000,2979.470000
3,fffe43004900440036003200370033003400,64,3.463645,3.481172,3.186039,3.51594,3.00000,3.447531,3.437031,3.468209,...,56,17,0.001757,0,27.57,3199,2,710,37554.530000,36064.590000
4,fffe43004900440035003000370031003900,16,3.463152,3.481172,4.494540,3.51594,4.00000,3.568330,3.514269,3.464717,...,57,53,0.002912,144,296.88,4123,1,685,40974.908351,40376.811649
